In [1]:
import huggingface_hub
huggingface_hub.login()

In [2]:
vae

AudioAutoencoder(
  (bottleneck): VAEBottleneck()
  (encoder): OobleckEncoder(
    (layers): Sequential(
      (0): Conv1d(2, 128, kernel_size=(7,), stride=(1,), padding=(3,))
      (1): EncoderBlock(
        (layers): Sequential(
          (0): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
              (2): SnakeBeta()
              (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
            )
          )
          (1): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (2): SnakeBeta()
              (3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
            )
          )
          (2): ResidualUnit(
            (layers): Sequential(
              (0): SnakeBeta()
              (1): Conv1d(128, 128, kernel_size=(7,), stride=(

In [1]:
import torch
import huggingface_hub
from stable_audio_tools import get_pretrained_model
import torchaudio

# 1. 모델 로드 (VAE 추출)
model, config = get_pretrained_model("stabilityai/stable-audio-open-1.0")
vae = model.pretransform.model  # 전체 모델에서 VAE 부분만 추출
vae.eval()  # 평가 모드로 전환

# 2. 더미 오디오 생성 (44.1kHz 스테레오, 2초)
# original_waveform = torch.randn(2, 44100 * 2)  # 무작위 노이즈 샘플
original_waveform = torchaudio.load("./OneShots (24bit WAV)/Kicks/RK_BT6_Kick_12.wav")[0]  # 실제 파일 사용 시

# 3. 인코딩 → 디코딩
with torch.no_grad():
    # 인코딩 (배치 차원 추가 필요)
    latent = vae.encode(original_waveform.unsqueeze(0))  # shape: [1, 64, 86]
    
    # 디코딩
    reconstructed_waveform = vae.decode(latent).squeeze(0)  # shape: [2, 88200]

# 4. 결과 확인
print(f"[차원 비교]")
print(f"원본: {original_waveform.shape} ({(original_waveform.shape[-1]/44100):.1f}초)")
print(f"잠재: {latent.shape} (압축률: {original_waveform.numel()/latent.numel():.1f}x)")
print(f"복원: {reconstructed_waveform.shape}")

# 5. 오디오 파일 저장
torchaudio.save(
    "reconstructed_audio.wav",
    reconstructed_waveform.cpu(),  # GPU 텐서일 경우 CPU로 이동
    sample_rate=config["sample_rate"]  # 44100 Hz
)


/home/sake/.local/share/virtualenvs/stable-audio-tools-c38r8sYS/lib/python3.10/site-packages/lightning_fabric/__init__.py:40: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
/home/sake/.local/share/virtualenvs/stable-audio-tools-c38r8sYS/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


[차원 비교]
원본: torch.Size([2, 75600]) (1.7초)
잠재: torch.Size([1, 64, 36]) (압축률: 65.6x)
복원: torch.Size([2, 73728])


In [7]:
original_waveform_2 = torchaudio.load("./OneShots (24bit WAV)/Kicks/RK_BT7_Kick_04.wav")[0]  # 실제 파일 사용 시

In [8]:
latent_2 = vae.encode(original_waveform_2.unsqueeze(0))

In [9]:
latent.shape, latent_2.shape

(torch.Size([1, 64, 36]), torch.Size([1, 64, 18]))

In [12]:
latent_2_padded = torch.nn.functional.pad(latent_2, (0, latent.shape[-1] - latent_2.shape[-1]))
latent_2_padded.shape

torch.Size([1, 64, 36])

In [14]:
l = (latent + latent_2_padded) / 2
l.shape

torch.Size([1, 64, 36])

In [15]:
out = vae.decode(l)
out.shape

torch.Size([1, 2, 73728])

In [17]:
torchaudio.save(
    "output.wav",
    out.cpu().squeeze(0),  # GPU 텐서일 경우 CPU로 이동
    sample_rate=config["sample_rate"]  # 44100 Hz
)

In [2]:
enc_out = vae.encoder(original_waveform.unsqueeze(0))

In [3]:
enc_out.shape

torch.Size([1, 128, 36])

In [4]:
latent.shape

torch.Size([1, 64, 36])

## Interactive

In [2]:
from pathlib import Path
from collections import defaultdict
from tqdm import tqdm

In [3]:
p = Path("OneShots (24bit WAV)")

In [4]:
samples = list(p.rglob("*.wav"))
len(samples)

77

In [ ]:
sample_dict = defaultdict(dict)
for s in tqdm(samples):
  loaded, sr = torchaudio.load(s)
  if loaded.shape[0] == 1:
    loaded = torch.cat([loaded, loaded])
  # Ensure loaded is exactly 2 seconds (2 * sr samples)
  target_length = 2 * sr
  if loaded.shape[1] > target_length:
    # Truncate if longer than 2 seconds
    loaded = loaded[:, :target_length]
  elif loaded.shape[1] < target_length:
    # Zero pad if shorter than 2 seconds
    pad_length = target_length - loaded.shape[1]
    loaded = torch.nn.functional.pad(loaded, (0, pad_length))
  latent = vae.encode(loaded.unsqueeze(0))
  sample_dict[s.parent.name][s.name] = latent

In [6]:
import json
import torch

# Convert sample_dict to JSON-serializable format
def save_sample_dict_to_json(sample_dict, filepath):
  json_dict = {}
  for category, files in sample_dict.items():
    json_dict[category] = {}
    for filename, latent in files.items():
      # Convert tensor to list for JSON serialization
      json_dict[category][filename] = latent.cpu().numpy().tolist()
  
  with open(filepath, 'w') as f:
    json.dump(json_dict, f, indent=2)
  print(f"Saved sample_dict to {filepath}")


# Save the current sample_dict
save_sample_dict_to_json(sample_dict, "sample_dict.json")

Saved sample_dict to sample_dict.json


In [5]:
import json
import torch

# Load sample_dict from JSON
def load_sample_dict_from_json(filepath):
  with open(filepath, 'r') as f:
    json_dict = json.load(f)
  
  sample_dict = {}
  for category, files in json_dict.items():
    sample_dict[category] = {}
    for filename, latent_list in files.items():
      # Convert list back to tensor
      sample_dict[category][filename] = torch.tensor(latent_list)
  
  print(f"Loaded sample_dict from {filepath}")
  return sample_dict

# Example of loading (uncomment to use)
sample_dict = load_sample_dict_from_json("sample_dict.json")

Loaded sample_dict from sample_dict.json


In [6]:
sample_dict["Kicks"]['RK_BTG2_Kick_13.wav'].shape

torch.Size([1, 64, 43])

In [7]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px

# 1. Flatten and collect data
latents = []
labels = []
filenames = []

for label, files in sample_dict.items():
  for fname, latent in files.items():
    # latent: torch.Size([1, 64, 43])
    flat = latent.flatten().cpu().numpy()
    latents.append(flat)
    labels.append(label)
    filenames.append(fname)

latents = np.stack(latents)  # shape: (N, 2752)

# 2. Dimensionality reduction (t-SNE, you can use UMAP if you prefer)
tsne = TSNE(n_components=3, random_state=42, perplexity=10)
latents_3d = tsne.fit_transform(latents)

# 3. Prepare DataFrame for Plotly
df = pd.DataFrame(latents_3d, columns=['x', 'y', 'z'])
df['label'] = labels
df['filename'] = filenames

# 4. Plotly 3D scatter plot
fig = px.scatter_3d(
  df, x='x', y='y', z='z',
  color='label',
  hover_name='filename',
  title='Latent Space Visualization'
)
fig.update_traces(marker=dict(size=5))
fig.show()

In [11]:
import plotly.io as pio
pio.renderers.default = 'vscode'

In [9]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, Audio

# 1. Flatten and collect data
latents = []
labels = []
filenames = []
latent_dict = {}

for label, files in sample_dict.items():
  for fname, latent in files.items():
    flat = latent.flatten().cpu().numpy()
    latents.append(flat)
    labels.append(label)
    filenames.append(fname)
    latent_dict[fname] = latent  # Store original latent for decoding

latents = np.stack(latents)

# 2. Dimensionality reduction
tsne = TSNE(n_components=3, random_state=42, perplexity=10)
latents_3d = tsne.fit_transform(latents)

# 3. Prepare DataFrame
df = pd.DataFrame(latents_3d, columns=['x', 'y', 'z'])
df['label'] = labels
df['filename'] = filenames

# 4. Interactive plot with selection
scatter = go.Scatter3d(
  x=df['x'], y=df['y'], z=df['z'],
  mode='markers',
  marker=dict(size=5, color=pd.factorize(df['label'])[0], colorscale='Viridis'),
  text=df['filename'],
  customdata=df['filename'],
  name='Latents'
)

fig = go.FigureWidget(data=[scatter])
fig.update_layout(title='Latent Space Visualization', scene=dict(aspectmode='cube'))

# 5. Widget for selected filename
selected_filename = widgets.Text(value='', description='Selected:', disabled=True)
play_button = widgets.Button(description='Play Selected')
output = widgets.Output()

def on_point_click(trace, points, selector):
  if points.point_inds:
    idx = points.point_inds[0]
    fname = df.iloc[idx]['filename']
    selected_filename.value = fname

scatter.on_click(on_point_click)

def on_play_clicked(b):
  with output:
    output.clear_output()
    fname = selected_filename.value
    if fname and fname in latent_dict:
      latent = latent_dict[fname]
      # 디코딩 (예시: vae.decode)
      with torch.no_grad():
        decoded = vae.decode(latent).squeeze().cpu().numpy()
      # decoded shape: (2, N) or (N,)
      # torchaudio expects (channels, samples), but Audio widget expects (samples,) or (samples, channels)
      if decoded.ndim == 2:
        decoded = decoded.T  # (samples, channels)
      display(Audio(decoded, rate=44100))  # sr을 실제 sr로 맞춰주세요
    else:
      print("No latent selected or not found.")

play_button.on_click(on_play_clicked)

# 6. Display everything
display(fig, selected_filename, play_button, output)

FigureWidget({
    'data': [{'customdata': array(['RK_BTG2_Impact_01.wav', 'RK_BTG2_Impact_05.wav',
                                   'RK_BTG2_Impact_09.wav', 'RK_DDG1_Perc_02.wav', 'RK_BTG2_Perc_02.wav',
                                   'RK_FMG3_Perc_07.wav', 'RK_BT7_Percusion_02.wav',
                                   'RK_BT7_Percusion_08.wav', 'RK_DM6_Perc_04.wav', 'RK_BT6_Perc_20.wav',
                                   'RK_BTG2_Perc_15.wav', 'RK_BT6_Perc_04.wav', 'RK_BTG2_Kick_13.wav',
                                   'RK_BT7_Kick_12.wav', 'RK_DDG1_Kick_15.wav', 'RK_DM6_Kick_11.wav',
                                   'RK_BTG2_Kick_05.wav', 'RK_BT7_Kick_07.wav', 'RK_FMG3_Kick_01.wav',
                                   'RK_BT6_Kick_12.wav', 'RK_FMG3_Kick_05.wav', 'RK_DM6_Kick_01.wav',
                                   'RK_BT7_Kick_04.wav', 'RK_DM6_Kick_04.wav', 'RK_DDG1_Kick_02.wav',
                                   'RK_DM6_Hihat_07.wav', 'RK_DDG1_Hihat_14.wav', 'RK_BTG2_

Text(value='', description='Selected:', disabled=True)

Button(description='Play Selected', style=ButtonStyle())

Output()

In [14]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, Audio

def safe_audio(decoded, sr=44100):
  # float32로 변환
  decoded = np.array(decoded, dtype=np.float32)
  # -1.0 ~ 1.0로 clip
  decoded = np.clip(decoded, -1.0, 1.0)
  return Audio(decoded, rate=sr)

# 1. Flatten and collect data
latents = []
labels = []
filenames = []
latent_dict = {}

for label, files in sample_dict.items():
  for fname, latent in files.items():
    flat = latent.flatten().cpu().numpy()
    latents.append(flat)
    labels.append(label)
    filenames.append(fname)
    latent_dict[fname] = latent

latents = np.stack(latents)

# 2. Dimensionality reduction
tsne = TSNE(n_components=3, random_state=42, perplexity=10)
latents_3d = tsne.fit_transform(latents)

# 3. Prepare DataFrame
df = pd.DataFrame(latents_3d, columns=['x', 'y', 'z'])
df['label'] = labels
df['filename'] = filenames

# 4. 3D plot (시각화만)
fig = px.scatter_3d(
  df, x='x', y='y', z='z',
  color='label',
  hover_name='filename',
  title='Latent Space Visualization'
)
fig.update_traces(marker=dict(size=5))
fig.show()

# 5. 드롭다운으로 latent 선택
dropdown = widgets.Dropdown(
  options=filenames,
  description='Select file:',
  style={'description_width': 'initial'},
  layout=widgets.Layout(width='50%')
)
play_button = widgets.Button(description='Play Selected')
output = widgets.Output()

def on_play_clicked(b):
  with output:
    output.clear_output()
    fname = dropdown.value
    if fname and fname in latent_dict:
      latent = latent_dict[fname]
      with torch.no_grad():
        decoded = vae.decode(latent).squeeze().cpu().numpy()
      # if decoded.ndim == 2:
      #   decoded = decoded.T
      display(safe_audio(decoded.squeeze(), sr=44100))  # sr을 실제 값으로 맞추세요
    else:
      print("No latent selected or not found.")

play_button.on_click(on_play_clicked)

display(dropdown, play_button, output)

Dropdown(description='Select file:', layout=Layout(width='50%'), options=('RK_BTG2_Impact_01.wav', 'RK_BTG2_Im…

Button(description='Play Selected', style=ButtonStyle())

Output()

In [22]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, Audio
import torch

# 기존 데이터 준비 코드는 그대로...
# (latents, labels, filenames, latent_dict 등)

# 3D plot 표시
fig = px.scatter_3d(
  df, x='x', y='y', z='z',
  color='label',
  hover_name='filename',
  title='Latent Space Visualization'
)
fig.update_traces(marker=dict(size=5))
fig.show()

# Linear Interpolation UI
print("=== Linear Interpolation ===")

# 두 파일 선택용 드롭다운
file1_dropdown = widgets.Dropdown(
  options=filenames,
  description='File 1:',
  style={'description_width': 'initial'},
  layout=widgets.Layout(width='40%')
)

file2_dropdown = widgets.Dropdown(
  options=filenames,
  description='File 2:',
  style={'description_width': 'initial'},
  layout=widgets.Layout(width='40%')
)

# Interpolation 슬라이더 (0.0 = File1, 1.0 = File2)
interp_slider = widgets.FloatSlider(
  value=0.5,
  min=0.0,
  max=1.0,
  step=0.05,
  description='Interpolation:',
  style={'description_width': 'initial'},
  layout=widgets.Layout(width='60%')
)

# 재생 버튼
play_interp_button = widgets.Button(description='Play Interpolated')
output_interp = widgets.Output()

def safe_audio(decoded, sr=44100):
  import numpy as np
  decoded = np.array(decoded, dtype=np.float32)
  # (channels, samples) -> (samples, channels)
  # if decoded.ndim == 2 and decoded.shape[0] < decoded.shape[1]:
  #   decoded = decoded.T
  # if decoded.ndim == 2 and decoded.shape[1] == 1:
  #   decoded = decoded[:, 0]
  decoded = np.clip(decoded, -1.0, 1.0)
  return Audio(decoded, rate=44100)

def on_play_interp_clicked(b):
  with output_interp:
    output_interp.clear_output()
    
    file1 = file1_dropdown.value
    file2 = file2_dropdown.value
    alpha = interp_slider.value
    
    if file1 and file2 and file1 in latent_dict and file2 in latent_dict:
      # 두 latent 가져오기
      latent1 = latent_dict[file1]
      latent2 = latent_dict[file2]
      
      # Linear interpolation: (1-alpha) * latent1 + alpha * latent2
      interpolated_latent = (1 - alpha) * latent1 + alpha * latent2
      
      # 디코딩
      with torch.no_grad():
        decoded = vae.decode(interpolated_latent).squeeze().cpu().numpy()
      
      print(f"Interpolating between '{file1}' and '{file2}' (α={alpha:.2f})")
      display(safe_audio(decoded.squeeze(), sr=44100))
    else:
      print("Please select valid files.")

play_interp_button.on_click(on_play_interp_clicked)

# UI 레이아웃
file_selection = widgets.HBox([file1_dropdown, file2_dropdown])
interp_controls = widgets.VBox([
  widgets.HTML("<b>Linear Interpolation Controls:</b>"),
  file_selection,
  interp_slider,
  play_interp_button,
  output_interp
])

display(interp_controls)

# 기존 단일 파일 재생 UI도 유지
print("\n=== Single File Playback ===")
dropdown = widgets.Dropdown(
  options=filenames,
  description='Select file:',
  style={'description_width': 'initial'},
  layout=widgets.Layout(width='50%')
)
play_button = widgets.Button(description='Play Selected')
output = widgets.Output()

def on_play_clicked(b):
  with output:
    output.clear_output()
    fname = dropdown.value
    if fname and fname in latent_dict:
      latent = latent_dict[fname]
      with torch.no_grad():
        decoded = vae.decode(latent).squeeze().cpu().numpy()
      display(safe_audio(decoded.squeeze(), sr=44100))
    else:
      print("No latent selected or not found.")

play_button.on_click(on_play_clicked)

single_controls = widgets.VBox([
  widgets.HTML("<b>Single File Playback:</b>"),
  dropdown,
  play_button,
  output
])

display(single_controls)

=== Linear Interpolation ===



=== Single File Playback ===
